In [54]:
import os

BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

Config set: http://localhost:8000/v1


In [55]:
!curl http://localhost:8000/v1/models -H "Authorization: Bearer $OPENAI_API_KEY"

{"object":"list","data":[{"id":"Qwen3-8B","object":"model","created":1752941441,"owned_by":"vllm","root":"Qwen/Qwen3-8B","parent":null,"max_model_len":8192,"permission":[{"id":"modelperm-ad2c06f72fa048ec8ecaab3c3dc00e62","object":"model_permission","created":1752941441,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [56]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE

server = server = MCPServerSSE('http://localhost:8090/sse') 

In [57]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = model = OpenAIModel("Qwen3-8B", provider=provider)

In [58]:
from pydantic_ai import Agent

agent = Agent(
    model=agent_model,
    toolsets=[server],
    system_prompt = """
    The tool to check for most optimal time frame:
    get_free_time_slots()
    Args:
    attendees (list): List of attendee identifiers (usernames, emails, or IDs)
                     whose calendars will be checked for availability.
    duration_minutes (int): Required meeting duration in minutes. Only time slots
                           with at least this much available time will be returned.
    start_date (str): Start of the search period in ISO format (YYYY-MM-DDTHH:MM:SS)
                     or similar datetime string format.
    end_date (str): End of the search period in ISO format (YYYY-MM-DDTHH:MM:SS)
                   or similar datetime string format.
 """
)

In [59]:
import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output


In [60]:
from mcp.server.fastmcp import FastMCP
from mcp.server import Server
import json
import logging
from typing import Optional, Dict, Any
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from mcp.server.sse import SseServerTransport
import argparse
from starlette.routing import Mount, Route
from pydantic import BaseModel
from starlette.applications import Starlette
from starlette.requests import Request
import uvicorn
import os
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE

In [61]:
import time

start_time = time.time()
result = await run_async("""
    Time Frame To Check:
    'start_timeframe': '2025-07-17T00:00:00+05:30',
    'end_timeframe': '2025-07-19T11:59:59+05:30',
    'Duration (min)': 60

    User's Involved In This Analysis: userone.amd@gmail.com, usertwo.amd@gmail.com and userthree.amd@gmail.com

    Check for the most optimal timeframe slot.
""")
end_time = time.time()

execution_time = end_time - start_time
print(f"Execution time: {execution_time:.3f} seconds")
print(f"Result: {result}")

UnexpectedModelBehavior: Tool 'get_free_time_slots' exceeded max retries count of 1

In [ ]:
# Define response model for meeting extraction
from pydantic import BaseModel
class MeetingTimeframe(BaseModel):
    start_timeframe: Optional[str] = None  # When to start looking for slots
    end_timeframe: Optional[str] = None    # When to stop looking for slots
    duration_minutes: Optional[int] = None  # Duration of the meeting in minutes


meeting_agent = Agent(
    model,
    output_type=MeetingTimeframe,
    system_prompt=f"""
    You are an expert at extracting meeting scheduling information from emails and messages.
    
    Current date, time and day: {current_date} - {datetime.now().strftime("%A")}
    
    Your task is to analyze the content and extract meeting scheduling parameters:
    
    1. start_timeframe: The earliest date to start looking for meeting slots
       - For "today": use current date
       - For "tomorrow": use next day
       - For day names (Monday, Tuesday, etc.): find the NEXT occurrence of that day
       - For "this week": use current date
       - For "next week": use date of next Monday
       - For specific dates: parse and use as-is
       - If no timeframe mentioned: default to current date
       - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    
    2. end_timeframe: The latest date to stop looking for meeting slots
       - If specific day mentioned: use same day as start_timeframe
       - If "this week" mentioned: use end of current week (Friday)
       - If "next week" mentioned: use end of next week (Friday)  
       - If range mentioned (e.g., "Monday to Wednesday"): use end of range
       - If no end specified: default to 7 days from start_timeframe
       - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    
    3. duration_minutes: Meeting duration in minutes
       - Parse explicit durations: "30 minutes", "1 hour", "2 hours"
       - Convert hours to minutes: 1 hour = 60 minutes
       - Common defaults: "quick chat" = 15 min, "brief meeting" = 30 min
       - If no duration specified: default to 60 minutes
    
    Examples:
    - "Let's meet Thursday" → start: next Thursday, end: same Thursday, duration: 60
    - "30 minute call this week" → start: today, end: this Friday, duration: 30
    - "2 hour session tomorrow" → start: tomorrow, end: tomorrow, duration: 120
    - "Meeting Monday to Wednesday" → start: next Monday, end: next Wednesday, duration: 60

    OUTPUT FORMAT EXAMPLE:
    
    "StartTime": "2025-07-20T10:30:00+05:30",
    "EndTime": "2025-07-20T11:00:00+05:30",
    
    Be precise with date calculations and always consider the current date context.
    """,
)

In [ ]:

# Get current date for context
current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Run the agent to extract meeting information
result = await meeting_agent.run("Let's meet on Monday for 1 hour")

# Convert to dictionary for JSON serialization
response_data = {
    "start_timeframe": result.data.start_timeframe,
    "end_timeframe": result.data.end_timeframe,
    "duration_minutes": result.data.duration_minutes,
    "success": True,
    "message": "Successfully extracted meeting timeframes"
}


In [ ]:
response_data

In [ ]:
current_date